## 1 - Objetivo

Neste exercício vamos explorar a biblioteca tensorflow para construir redes neurais artificiais. A intenção é observar como uma biblioteca que contém funções prontas pode nos auxiliar a criar nossos models de maneira mais rápida.

Novamente utilizaremos a classificação de imagens de números do dataset [MNIST](http://yann.lecun.com/exdb/mnist/) como exemplo. Desta forma, poderemos comparar o código gerado neste exercício com o código do exercício anterior.



## 2 - Carregando as bibliotecas

[Tensorflow](https://www.tensorflow.org/) é uma biblioteca de redes neurais para python desenvolvido inicialmente pelo Google. Atualmente é *open source* e possui uma base ativa de desenvolvimento.

Além desta biblioteca, vamos utilizar um módulo chamado ```input_data``` criado especialmente para o propósito deste exercício.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import tensorflow as tf

import tensorflow.contrib.eager as tfe
import input_data

### 3 - Funções auxiliares

Neste exercício, faremos uso de diversas funções auxiliares para melhor visualizar cada etapa de otimização da rede neural.

Primeiro, temos uma função que vai carregar o dataset. A maior parte da lógica vem do módulo ```input_data```. A função vai retornar os datapoints divididos em dataset de treino e de teste.

In [ ]:
def load_data(data_dir):
  """Returns training and test tf.data.Dataset objects."""
  data = input_data.read_data_sets(data_dir, one_hot=True)
  train_ds = tf.data.Dataset.from_tensor_slices((data.train.images,
                                                 data.train.labels))
  test_ds = tf.data.Dataset.from_tensors((data.test.images, data.test.labels))
  return (train_ds, test_ds)

Agora, vamos definir a função de custo que será utilizada para calcular o sinal de erro do algoritmo backpropagation.

In [ ]:
def loss(predictions, labels):
  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(
#       tf.nn.softmax_cross_entropy_with_logits(
          logits=predictions, labels=labels))

Aqui definimos a função que vai calcular o percentual de acertos obtidos pela nossa rede neural.

In [ ]:
def compute_accuracy(predictions, labels):
  return tf.reduce_sum(
      tf.cast(
          tf.equal(
              tf.argmax(predictions, axis=1,
                        output_type=tf.int64),
              tf.argmax(labels, axis=1,
                        output_type=tf.int64)),
          dtype=tf.float32)) / float(predictions.shape[0].value)

A próxima função é aquela que irá executar a lógica de otimização da rede neural. Nela, nós temos que definir a lógica de gradiente descendente e backpropagation. Note como o código é enxuto. 

Para entender como ele funciona, atente aos comentários do código.


In [ ]:
def train_one_epoch(model, optimizer, dataset, log_interval=None):
  """Treina o modelo usando um 'dataset' com  um 'otimizador'."""

  #  esta linha serve para ativar ou buscar um contador interno da biblioteca que retorna o número do 
  # 'batch' executado - se existe, busca o existente, senão cria um novo
  tf.train.get_or_create_global_step()  
  
  # este é o laço principal da otimização
  for (batch, (inputs, alvos)) in enumerate(tfe.Iterator(dataset)):
    # esta linha diz para a biblioteca guardar o estado da otimização a cada 10 'batches' contados por
    # tf.train.get_or_create_global_step() criado acima
    # desta forma, podemos visualizar o comportamento da otimização e entendermos como o modelo está funcionando e
    # quais partes podem ser melhoradas
    with tf.contrib.summary.record_summaries_every_n_global_steps(10):
      # nesta linha, definimos uma 'fita' que vai armazenar a ordem de execução da rede neural e assim como
      # os gradientes calculados para backpropagation
      with tfe.GradientTape() as tape:
        prediction = model(inputs, training=True)  # obtendo a saída da rede neural
        loss_value = loss(prediction, alvos)  # calculando o sinal de erro
        tf.contrib.summary.scalar('loss', loss_value)  # aqui, adicionamos o sinal de erro à vizualição
        tf.contrib.summary.scalar('accuracy',
                                  compute_accuracy(prediction, alvos)) # adicionamos também o percentual de acertos
      # é aqui que a 'mágica' acontece
      # tensorflow possui um módulo de auto-diferenciação que lê a 'fita' de execução e calcula todos as derivadas 
      # necessárias para o algoritmo de backpropagation, incluindo a execução do próprio algoritmo!
      # desta forma, não precisamos nos preocupar com a lógica daquele algoritmo e podemos nos concentrar
      # no modelo em si
      grads = tape.gradient(loss_value, model.variables) 
      # uma vez executado o backpropagation, o optimizer irá aplicar a atualização de parâmetros
      optimizer.apply_gradients(zip(grads, model.variables))
      # por fim, vamos imprimir na tela o progresso da otimização
      if log_interval and batch % log_interval == 0:
        print('Batch #%d\tLoss: %.6f' % (batch, loss_value))

Após criarmos o laço de otimização, é a vez de criarmos o laço de teste. Assim como no laboratório anterior, não precisamos de uma lógica de backpropagation, apenas uma forma de calcular a performance da rede neural. O código é juito similar ao código de otimização.

Para mais detalhes, veja os comentários do código.

In [ ]:
def test(model, dataset):
  """Realiza um teste do 'model' utilizando datapoins retirados do 'dataset'."""
  avg_loss = tfe.metrics.Mean('loss')  # aqui usamos uma função pré-construída para calcular a média do sinal de erro
  accuracy = tfe.metrics.Accuracy('accuracy')  # e fazemos o mesmo com o percentual de acertos

  # este é o laço principal do teste
  for (images, labels) in tfe.Iterator(dataset):
    predictions = model(images, training=False)  # obtemos as predições
    avg_loss(loss(predictions, labels))  # calculamos a média do sinal de erro
    accuracy(tf.argmax(predictions, axis=1, output_type=tf.int64),  
             tf.argmax(labels, axis=1, output_type=tf.int64))  # calculamos o percentual de acertos
  print('Test set: Average loss: %.4f, Accuracy: %4f%%\n' %
        (avg_loss.result(), 100 * accuracy.result()))  # imprimimos na tela algumas dessas informações

  # por fim, nas linhas abaixo, adicionamos mais dados à visualização do treino
  with tf.contrib.summary.always_record_summaries():
    tf.contrib.summary.scalar('loss', avg_loss.result())
    tf.contrib.summary.scalar('accuracy', accuracy.result())

### 4 - Grafos e * eager execution*

De maneira geral, cada vez que executamos um código escrito usando a biblioteca tensorflow, seguimos uma lista de passos:

1. Carregamos o(s) dataset(s).
2. Definimos o modelo que será treinado.
3. Treinamos o modelo.
4. Avaliamos sua performance.
5. Usamos o modelo para fazer previsões/classificações.

Esta biblioteca possui interfaces de alto e baixo nível que nos dão flexibilidade na hora de escrevermos o código do nosso modelo. Na maioria das vezes podemos utilizar interfaces de alto nível para as etapas 1, 3, 4 e 5. O ponto principal de variação se encontra na etapa 2.

A biblioteca nos oferece 2 formas de executar os programas: através de grafos (preferida pela biblioteca) ou de forma imperativa (chamado pela biblioteca de modo *eager execution*).

Na forma de grafos, nós precisamos primeiro definir a dependência de funções que compõem a rede neural (*i.e.*, as camadas). Uma vez feito isso, a biblioteca compila um programa que então será usado para treinar o modelo. Esta forma de definir a rede neural faz com que diversas otimizações internas (feitas pela própria biblioteca) sejam possíveis, especialmente na parte de auto-diferenciação e *backpropagation*, fazendo com que o programa seja executado mais rapidamente. 

No entanto, é demasiado difícil depurarmos um programa escrito desta forma. Nós não temos acesso ao grafo de dependências construído e precisamos encontrar eventuais erros de lógica usando muita intuição ou fazendo testes de mesa. Por isso, os mantenedores da biblioteca criaram uma forma diferente de definirmos nossa rede neural: modo *eager execution*.

O modo *eager execution* segue uma forma imperativa, mais tradicional, de programação. Nós podemos criar a função e executá-la imediatamente, tendo assim, uma forma de acompanhar o fluxo do nosso código e facilitar a depuração. Para isso, a biblioteca cria uma "fita" de dependências que é escrita, lida e apagada a cada iteração. Desta forma, diversas otimizações que podem ser feitas na execução por grafos não são possíveis com *eager execution*.

De maneira geral, recomenda-se iniciar a codificação de nossos modelos utilizando *eager execution* e, uma vez que tenhamos certeza de que tudo funciona como devido, trocarmos para o modo de grafos.


#### 4.1 - Definindo a rede neural

Abaixo vamos definir uma rede neural para classificar os dígitos do dataset MNIST. Esta estrutura é similar à rede LeNet5 de 1998, utilizada pelos correios dos Estados Unidos para auxiliar na leitura dos códigos de endereçamento postal. 

Esta rede possui 7 camadas:

1. Input
2. Convolução 1
3. Maxpool 1
4. Convolução 2
5. MaxPool 2
6. ReLu (aqui está a principal diferença em relação a original, que usava uma camada sigmoid)
7. Linear (isto é, não possui uma função de ativação, apenas a transformação linear)

Para detalhes, veja os comentários no código abaixo. Para uma visualização esquemática da rede, veja os slides do Dia 04, seção "Redes Neurais Convolucionais".

In [ ]:
class MNISTModel(tfe.Network):
    """MNIST Network.
    
        Para podermos executar nosso modelo no modo eager execution, nossa classe deve obrigatoriamente 
            herdar da classe tfe.Network.
    
    """
    
    def __init__(self, data_format):
        super(MNISTModel, self).__init__(name='')
        # esta parte define o formato dos dados de input - recomenda-se utilizar o padrão definido no código
        # de treino - para descrições mais detalhadas, veja a documentação da classe tf.layers.Conv2D
        if data_format == 'channels_first':
            self._input_shape = [-1, 1, 28, 28]
        else:
            assert data_format == 'channels_last'
            self._input_shape = [-1, 28, 28, 1]
        # neste ponto definimos a estrutura da rede neural
        # para que a 'fita" de gradientes mantenha um registro das dependências e a biblioteca mantenha
        # um registro dos parâmetros, devemos passar a definição de cada camada para a função self.track_layer
        # que é definida em tfe.Network
        # perceba que cada camada é definida por uma classe que pertence ao módulo tf.layers
        # para as camadas Conv2d (Convoluções), nós precisamos definir o número de filtros e o tamanho do filtro -
        # para definir o tamanho do filtro podemos passar um número inteiro (define mesma altura /largura) ou uma 
        # tupla que definirá altura/largura customizados
        self.conv1 = self.track_layer(
            tf.layers.Conv2D(filters=32, kernel_size=5, data_format=data_format, activation=tf.nn.relu))
        self.conv2 = self.track_layer(
            tf.layers.Conv2D(filters=64, kernel_size=5, data_format=data_format, activation=tf.nn.relu))
        # para definir as camadas tradicionais, selecionamos a camada Dense e definimos o número de neurônios (units)
        # na camada, assim como a função de ativação (activation) - no caso de não passarmos uma função de ativação 
        # como parâmetro, a camada se torna uma camada 'linear'- em outras palavras, a camada realiza apenas o cálculo
        # da transformação linear sobre os inputs
        self.fc1 = self.track_layer(tf.layers.Dense(units=1024, activation=tf.nn.relu))
        self.fc2 = self.track_layer(tf.layers.Dense(units=10))
        self.dropout = self.track_layer(tf.layers.Dropout(0.5))
        # a camada de maxpool não possui parâmetros pois ela apelas realiza uma operação 'max' (seleciona o maior 
        # valor dentre uma lista de valores) e, desta forma, podemos criar apenas uma camada em nosso código e 
        # reutilizá-la para cada Conv2d. para definir esta camada, devemos definir o tamanho da 'janela' de valores e 
        # o deslocamento da 'janela' sobre o resultado da convolução. a definição da altura/largura segue o mesmo 
        # padrão utilizado na camada Conv2d
        self.max_pool2d = self.track_layer(
            tf.layers.MaxPooling2D(
                pool_size=(2, 2), strides=(2, 2), padding='SAME', data_format=data_format))

    def call(self, inputs, training):
        """ Classifica os dígitos baseado nos 'inputs'.
        """
        # para termos certeza de que os inputs estão no formato esperado, realizamos uma operação reshape
        x = tf.reshape(inputs, self._input_shape)
        # em seguida, aplicamos em ordem as camadas definidas na inicialização da classe
        x = self.conv1(x)  # primeira convolução
        x = self.max_pool2d(x)  # primeiro max-pool
        x = self.conv2(x)  # segunda convolução
        x = self.max_pool2d(x)  # segundo max-pool - nota: esta é a mesma função acima - ver comentários na definição
        # como nós estamos operando sobre imagens, cada input será 2d (uma matriz). no entanto, as camadas 
        # Dense (camadas 'tradicionais') esperam um input 1d para cada exemplo. por isso, nós usamos uma função 
        # pré-contruida para fazer um reshape e transformar o input 2d em 1d - nota: nós poderíamos ter utilizado
        # a função reshape como feito anteriormente, passando o formato correto.
        x = tf.layers.flatten(x)
        x = self.fc1(x)  # primeira camada dense
        # aqui aplicamos uma função de dropout para auxiliar na regularização - o parâmetro training serve para
        # ativar/desativar o dropout - ele é aplicado apenas durante o treino e não durante o teste
        x = self.dropout(x, training=training) 
        x = self.fc2(x)  #  segunda camada dense (output)
        return x


#### 4.2 - O programa de treino

Neste exercício nós vamos utilizar o modo *eager execution*  para observarmos e alterarmos o comportamento de nossa
rede neural. O código para treinar a rede neural está definido abaixo, fazendo uso das funções auxiliares que criamos anteriormente.

Para mais detalhes, atente aos comentários no código abaixo.

In [ ]:
def main(_):
  
    # primeiro devemos habilitar o modo eager execution
    # aviso: este comando deve ser executado apenas uma vez. caso contrário um exceção será lançada
    tfe.enable_eager_execution()
    
    # aqui define-se o dispositivo que será utilizado para o treino da rede e o formato dos dados passados
    # de maneira geral, usamos o formato de dados default como definidos abaixo
    (device, data_format) = ('/gpu:0', 'channels_first')
    if FLAGS.no_gpu or tfe.num_gpus() <= 0:
        (device, data_format) = ('/cpu:0', 'channels_last')
        print('Using device %s, and data format %s.' % (device, data_format))

    # carregando os datasets em treino e teste e embaralhando os exemplos
    # embaralhar os exemplos de treino auxilia a "quebrar" as dependências criadas pelo processamento
    # ordenado do dataset
    (train_ds, test_ds) = load_data(FLAGS.data_dir)
    train_ds = train_ds.shuffle(60000).batch(FLAGS.batch_size)

    # aqui nós criamos a classe contendo o modelo - veja detalhes na seção anterior
    model = MNISTModel(data_format)    
    # aqui define-se o algoritmo que fará a otmização do modelo
    optimizer = tf.train.MomentumOptimizer(FLAGS.lr, FLAGS.momentum)
    
    # criamos um diretório para armazenar o modelo treinado
    if FLAGS.output_dir:
        train_dir = os.path.join(FLAGS.output_dir, 'train')
        test_dir = os.path.join(FLAGS.output_dir, 'eval')
        tf.gfile.MakeDirs(FLAGS.output_dir)
    else:
        train_dir = None
        test_dir = None
    # aqui definimos o local no qual o programa armazenará os resumos gerados sobre o treino e sobre o teste
    # assim como instância a classe que vai realizar a gravação dos resumos
    summary_writer = tf.contrib.summary.create_file_writer(train_dir, flush_millis=10000)
    test_summary_writer = tf.contrib.summary.create_file_writer(test_dir, flush_millis=10000, name='test')
    checkpoint_prefix = os.path.join(FLAGS.checkpoint_dir, 'ckpt')
    
    # esta é a parte principal do programa
    # primeiro selecionamos o dispositivo que será usado para o treino - CPU ou GPU
    with tf.device(device):
        # criamos um laço para controlar as épocas
        for epoch in range(1, 11):
            # a biblioteca nos fornece diversas funções auxiliares pré-construídas para facilitar 
            # a criação dos modelos. no caso abaixo, utilizamos uma função exclusiva do modo eager execution
            # que verifica se temos um modelo já salvo e utiliza para inicializar os parâmetros - pesos - da rede
            # isso é bastante útil quando o programa para no meio do treino e precisamos reiniciar a partir de um
            # certo ponto. no entanto, se temos um modelo salvo e queremos reiniciar do início, precisamos
            # excluir o modelo antigo
            with tfe.restore_variables_on_create(tf.train.latest_checkpoint(FLAGS.checkpoint_dir)):
                #  esta linha serve para ativar ou buscar um contador interno da biblioteca que retorna o número do 
                # 'batch' executado - se existe, busca o existente, caso contrário cria um novo
                global_step = tf.train.get_or_create_global_step()
                start = time.time()
                # utilizando a classe que grava os resumos do treino
                with summary_writer.as_default():
                    # executamos uma época de treino - ver código das funções auxiliares
                    train_one_epoch(model, optimizer, train_ds, FLAGS.log_interval)
                end = time.time()
                # feito isso, imprimimos algumas informações na tela para facilitar o acompanhamento
                print('\nTrain time for epoch #%d (global step %d): %f' % (
                    epoch, global_step.numpy(), end - start))
                # utilizando a classe que grava os resumos do teste
                with test_summary_writer.as_default():
                    # executamos uma época de verificação sobre o dataset de teste
                    test(model, test_ds)
                    # criamos uma lista com todos os parâmetros do modelo (e do laço de treino) que estamos treinando
                    all_variables = (model.variables + optimizer.variables() + [global_step])
                    # utilizando mais uma função utilitária (exclusiva do modo eager execution),
                    # salvamos em disco os parâmetros do modelo que estamos treinando
                    tfe.Saver(all_variables).save(checkpoint_prefix, global_step=global_step)

#### 4.3 - Treinando a rede neural


Abaixo temos o código que vai instanciar o programa de treino e otimizar a rede neural. Cada ```parser.add_argument``` adiciona um parâmetro a função do código que vai executar o treino da rede neural (não confundir com os parâmetros - pesos - da rede neural em si).

*Observação:* O código abaixo foi feito para rodar em um console python e não em um notebook. Por isso, se precisarmos alterar algum dos parâmetros devemos fazê-lo diretamente no código que adiciona aquele parâmetro.


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--data-dir',
    type=str,
    default='/tmp/tensorflow/mnist/input_data',
    help='Directory for storing input data')
parser.add_argument(
    '--batch-size',
    type=int,
    default=64,
    metavar='N',
    help='input batch size for training (default: 64)')
parser.add_argument(
    '--log-interval',
    type=int,
    default=10,
    metavar='N',
    help='how many batches to wait before logging training status')
parser.add_argument(
    '--output_dir',
    type=str,
    default=None,
    metavar='N',
    help='Directory to write TensorBoard summaries')
parser.add_argument(
    '--checkpoint_dir',
    type=str,
    default='/tmp/tensorflow/mnist/checkpoints/',
    metavar='N',
    help='Directory to save checkpoints in (once per epoch)')
parser.add_argument(
    '--lr',
    type=float,
    default=0.01,
    metavar='LR',
    help='learning rate (default: 0.01)')
parser.add_argument(
    '--momentum',
    type=float,
    default=0.5,
    metavar='M',
    help='SGD momentum (default: 0.5)')
parser.add_argument(
    '--no-gpu',
    action='store_true',
    default=False,
    help='disables GPU usage even if a GPU is available')

FLAGS, unparsed = parser.parse_known_args()
tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

### 5 - Desafios

Para exercitarmos a utilização da biblioteca, vamos propor alguns desafios:

1. Adicione/remova camadas Conv2d/Dense e note como os resultados da classificação mudam.
2. Utilize um algoritmo de treino diferente (sugestões: Adam, Adadelta ou Adagrad) e note como o valor de 'Loss' se comporta com estes algoritmos.
3. Usando o dataset de teste, implemente o algoritmo de paciência. *Observação*: na prática, nós usamos um dataset de validação para implementar o algoritmo de paciência. 
4. Acesse a documentação [tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers) e tente adicionar conexões residuais (*residual connections*) e normalização de batch (*batch normalization*) ao seu modelo.

